In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, expr
from elasticsearch import Elasticsearch

spark = SparkSession.builder \
    .appName("StreamStock") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.elasticsearch:elasticsearch-spark-30_2.12:7.15.1") \
    .config("es.nodes", "elasticsearch") \
    .config("es.port", "9200") \
    .config("es.nodes.wan.only", "false") \
    .config("es.index.auto.create", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
es = Elasticsearch(hosts='http://elasticsearch:9200')
# es.indices.create(index="realtime_stocks_ssi")
df = spark.readStream.format('kafka') \
    .option('kafka.bootstrap.servers', 'kafka-1:9092,kafka-2:9092, kafka-3:9092') \
    .option('subscribe', 'realtimeStockData') \
    .load()

df.printSchema()

df = df.withColumn('symbol', (split(df['value'], ',').getItem(0)))
df = df.withColumn('price', (split(df['value'], ',').getItem(1)))
df = df.withColumn('volume', (split(df['value'], ',').getItem(2)))
df = df.withColumn('cp', (split(df['value'], ',').getItem(3)))
df = df.withColumn('rcp', (split(df['value'], ',').getItem(4)))
df = df.withColumn('ba', (split(df['value'], ',').getItem(5)))
df = df.withColumn('sa', (split(df['value'], ',').getItem(6)))
df = df.withColumn('hl', (split(df['value'], ',').getItem(7)))
df = df.withColumn('pcp', (split(df['value'], ',').getItem(8)))
df = df.withColumn('time', (split(df['value'], ',').getItem(9)))

def save_data(df, batch_id):
    data = df.collect()
    index = 0
    for row in data:
        index = index + 1
        # doc_send = {"symbol": "{}".format(row['symbol']), "price": "{}".format(row['price']), "volume": "{}".format(row['volume']),
        #             "cp": "{}".format(row['cp']), "rcp": "{}".format(row['rcp']),
        #             "ba": "{}".format(row['ba']), "sa": "{}".format(row['sa']), "hl": "{}".format(row['hl']),
        #             "pcp": "{}".format(row['pcp']), "time": "{}".format(row['time'])}
        doc_send = {
            "symbol": row['symbol'],
            "price": float(row['price']),
            "volume": int(row['volume']),
            "cp": float(row['cp']),
            "rcp": float(row['rcp']),
            "ba": float(row['ba']),
            "sa": float(row['sa']),
            "hl": bool(row['hl']),
            "pcp": float(row['pcp']),
            "time": "{}".format(row['time'])
        }
        es.index(index="realtime_stocks_ssi", document=doc_send)
        print(doc_send)
df.writeStream.foreachBatch(save_data).start().awaitTermination()
# df.writeStream.format('console').outputMode("append").start().awaitTermination()
# doc['your_string_field.keyword'].value != null ? Float.parseFloat(doc['your_string_field.keyword'].value) : 0





:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f2b41639-a7b0-46b6-afc3-024d0d19f153;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in centr

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



24/12/29 13:13:10 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-87a5c8eb-678b-4bbd-8192-12aad33dc24d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/29 13:13:10 ERROR MicroBatchExecution: Query [id = 752fe1c1-67fa-40e1-92f3-77f6ecb0b771, runId = 0047192d-b88b-43b0-85ea-8d358b7e3106] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/sql/connector/read/streaming/SupportsTriggerAvailableNow
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1017)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:174)
	at java.base/java.net.URLClassLoader.defineClass(URLClassLoader.java:550)
	at java.base/java.net.URLClassLoader$1.run(URLCl

StreamingQueryException: org/apache/spark/sql/connector/read/streaming/SupportsTriggerAvailableNow
=== Streaming Query ===
Identifier: [id = 752fe1c1-67fa-40e1-92f3-77f6ecb0b771, runId = 0047192d-b88b-43b0-85ea-8d358b7e3106]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [2]:
!pip install elasticsearch


     |████████████████████████████████| 571 kB 70 kB/s             
     |████████████████████████████████| 64 kB 83 kB/s              
